In [54]:
import torch
from transformers import BertForSequenceClassification, AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
import torch.nn.functional as F
import pandas as pd
from transformers import AdamW
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from bertviz import model_view
from lime.lime_text import LimeTextExplainer
import numpy as np

In [21]:
model = AutoModelForSequenceClassification.from_pretrained("app/resources/bert_classification", output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english", do_lower_case=True)

In [24]:
inputs = tokenizer.encode("European stock markets are seen edging higher at the open Friday", return_tensors='pt')
outputs = model(inputs)
attention = outputs[-1]  # Output includes attention weights when output_attentions=True
tokens = tokenizer.convert_ids_to_tokens(inputs[0]) 
model_view(attention, tokens)

<IPython.core.display.Javascript object>

In [128]:
def predictor(text):
        examples = []
        for example in text:
            examples.append(tokenizer.encode(example, return_tensors='pt'))

        results = []
        for example in examples:
            with torch.no_grad():
                outputs = model(example)
            logits = outputs[0]
            logits = F.softmax(logits, dim = 1)
            logits.tolist()
            results.append(logits.cpu().detach().numpy()[0].tolist())

        return np.array(results)

In [137]:
x = predictor(["European stock markets are seen edging higher at the open Friday"])
type(x)

numpy.ndarray

In [135]:
label_names = ["negative", "positive"]
explainer = LimeTextExplainer(class_names=label_names)

In [136]:
exp = explainer.explain_instance("European stock markets are seen edging higher at the open Friday", get_prediction, num_features=1)
words = exp.as_list()

KeyboardInterrupt: 